In [6]:
import os
import glob
import pandas as pd 
import zipfile
import geopandas as gpd
import geoparquet as gpq
from utils import *
from congestion_metrics import *
import logging
# from simpledbf import Dbf5s

In [7]:
# SCEN_MAP = SCENARIO_P2_1_MAP.copy()
SCEN_MAP = SCENARIO_P1_MAP.copy()
list(SCEN_MAP.values())
PERIOD_MAP

suffix = '_p1'
# suffix = ''

In [8]:
def make_time_ijk(temp_df):

    time_df = temp_df.iloc[:20]
    vol_df = temp_df.iloc[22:42]

    time_ijk = pd.melt(time_df, id_vars='origin', var_name='destination', value_name='value')
    time_ijk['metric'] = 'Time'
    vol_ijk = pd.melt(vol_df, id_vars='origin', var_name='destination', value_name='value')
    vol_ijk['metric'] = 'Vol'  

    return (pd.concat([time_ijk, vol_ijk]))

# zip_path = '../inputs/agg_mx/p21_011_Time_GC_Agg.zip'
zip_path = '../inputs/agg_mx/p10_011_Time_GC_Agg.zip'

time_ijks = []

trippurpose_list = ['HBW', 'EB', 'Other', 'All']
mode_list = ['Car', 'PT']

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    time_csv_list = [x for x in file_list if "Time" in x.split("/")[-1]]
    gc_csv_list = [x for x in file_list if "MSM_GC" in x.split("/")[-1]]

    for csvf in time_csv_list:
        for scenario in list(SCEN_MAP.values()):
            if scenario >= 16:
                for period in PERIOD_MAP:
                    period_no = PERIOD_MAP[period]
                    scenario_no = "26" + str(period_no) + str(scenario)


                    for trippurpose in trippurpose_list:
                        for mode in mode_list:
                            if (scenario_no in csvf) and (mode in csvf) and (trippurpose in csvf): 
                                with zip_ref.open(csvf) as f:
                                    temp_df = pd.read_csv(f, skiprows=17).rename(columns={'       ':'origin'})
                                    combined_ijk = make_time_ijk(temp_df)
                                    combined_ijk['scenario'] = scenario_no
                                    combined_ijk['mode'] = mode
                                    combined_ijk['trippurpose'] = trippurpose
                                    combined_ijk['csv_no'] = csvf.split('/')[-1].split('_')[0]
                                    
                                    time_ijks.append(combined_ijk)

                    if (scenario_no in csvf) and ('HCV' in csvf):
                        with zip_ref.open(csvf) as f:
                            temp_df = pd.read_csv(f, skiprows=17).rename(columns={'       ':'origin'})
                            combined_ijk = make_time_ijk(temp_df)
                            combined_ijk['scenario'] = scenario_no
                            combined_ijk['mode'] = 'HCV'
                            combined_ijk['trippurpose'] = 'HCV'
                            combined_ijk['csv_no'] = csvf.split('/')[-1].split('_')[0]

                            time_ijks.append(combined_ijk)

time_ijk = pd.concat(time_ijks)
time_ijk['origin'] = time_ijk['origin'].str.strip()
time_ijk['destination'] = time_ijk['destination'].str.strip()
# gc_df.columns

In [9]:
def make_time_ijk(temp_df):

    gc_df = temp_df.iloc[:20]
    vol_df = temp_df.iloc[22:42]

    gc_ijk = pd.melt(gc_df, id_vars='origin', var_name='destination', value_name='value')
    gc_ijk['metric'] = 'GC'
    vol_ijk = pd.melt(vol_df, id_vars='origin', var_name='destination', value_name='value')
    vol_ijk['metric'] = 'Vol'  

    return (pd.concat([gc_ijk, vol_ijk]))

gc_ijks = []

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    gc_csv_list = [x for x in file_list if "MSM_GC" in x.split("/")[-1]]

    for csvf in gc_csv_list:
        for scenario in list(SCEN_MAP.values()):
            if scenario >= 16:
                for period in PERIOD_MAP:
                    period_no = PERIOD_MAP[period]
                    scenario_no = "26" + str(period_no) + str(scenario)


                    for trippurpose in trippurpose_list:
                        for mode in mode_list:
                            if (scenario_no in csvf) and (mode in csvf) and (trippurpose in csvf): 
                                with zip_ref.open(csvf) as f:
                                    temp_df = pd.read_csv(f, skiprows=17).rename(columns={'       ':'origin'})
                                    combined_ijk = make_time_ijk(temp_df)
                                    combined_ijk['scenario'] = scenario_no
                                    combined_ijk['mode'] = mode
                                    combined_ijk['trippurpose'] = trippurpose
                                    combined_ijk['csv_no'] = csvf.split('/')[-1].split('_')[0]
                                    
                                    gc_ijks.append(combined_ijk)

                    if (scenario_no in csvf) and ('HCV' in csvf):
                        with zip_ref.open(csvf) as f:
                            temp_df = pd.read_csv(f, skiprows=17).rename(columns={'       ':'origin'})
                            combined_ijk = make_time_ijk(temp_df)
                            combined_ijk['scenario'] = scenario_no
                            combined_ijk['mode'] = 'HCV'
                            combined_ijk['trippurpose'] = 'HCV'
                            combined_ijk['csv_no'] = csvf.split('/')[-1].split('_')[0]

                            gc_ijks.append(combined_ijk)

gc_ijk = pd.concat(gc_ijks)
gc_ijk['origin'] = gc_ijk['origin'].str.strip()
gc_ijk['destination'] = gc_ijk['destination'].str.strip()

In [10]:
time_ijk_2 = time_ijk[time_ijk['metric']=='Time']
vol_ijk = time_ijk[time_ijk['metric']=='Vol']
gc_ijk_2 = gc_ijk[gc_ijk['metric']=='GC'] 

In [11]:
print(time_ijk_2['value'].str.strip().astype(float).sum())
print(gc_ijk_2['value'].str.strip().astype(float).sum())
print(vol_ijk['value'].str.strip().astype(float).sum())

8922204.472176
15388964.198951999
99865805.12449181


In [12]:
ON_COLS = ['origin', 'destination', 'scenario', 'mode', 'trippurpose']

time_gc_ijk = time_ijk_2[ON_COLS + ['value']].rename(columns={'value':'Time'}).merge(gc_ijk_2[ON_COLS + ['value']].rename(columns={'value':'GC'}), on=ON_COLS, how='outer'
                                                  ).merge(vol_ijk[ON_COLS + ['value']].rename(columns={'value':'Vol'}), on=ON_COLS, how='outer')

In [13]:
time_ijk.to_csv(f'../outputs/time_gc/time_sector_ijk{suffix}.csv', index=None)
gc_ijk.to_csv(f'../outputs/time_gc/gc_sector_ijk.csv{suffix}', index=None)
time_gc_ijk.to_csv(f'../outputs/time_gc/time_gc_sector_ijk{suffix}.csv', index=None)

In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    gc_csv_list = [x for x in file_list if "MSM_GC" in x.split("/")[-1]]

    for csvf in gc_csv_list:
        for scenario in list(SCEN_MAP.values()):
            if scenario >= 16:
                for period in PERIOD_MAP:
                    period_no = PERIOD_MAP[period]
                    scenario_no = "26" + str(period_no) + str(scenario)


                    for trippurpose in trippurpose_list:
                        for mode in mode_list:
                            if (scenario_no in csvf) and (mode in csvf) and (trippurpose in csvf): 
                                with zip_ref.open(csvf) as f:
                                    temp_df = pd.read_csv(f, skiprows=17).rename(columns={'       ':'origin'})
                                    combined_ijk = make_time_ijk(temp_df)
                                    combined_ijk['scenario'] = scenario_no
                                    combined_ijk['mode'] = mode
                                    combined_ijk['trippurpose'] = trippurpose
                                    combined_ijk['csv_no'] = csvf.split('/')[-1].split('_')[0]
                                    
                                    gc_ijks.append(combined_ijk)

                    if (scenario_no in csvf) and ('HCV' in csvf):
                        with zip_ref.open(csvf) as f:
                            temp_df = pd.read_csv(f, skiprows=17).rename(columns={'       ':'origin'})
                            combined_ijk = make_time_ijk(temp_df)
                            combined_ijk['scenario'] = scenario_no
                            combined_ijk['mode'] = 'HCV'
                            combined_ijk['trippurpose'] = 'HCV'
                            combined_ijk['csv_no'] = csvf.split('/')[-1].split('_')[0]

                            gc_ijks.append(combined_ijk)

gc_ijk = pd.concat(gc_ijks)
gc_ijk['origin'] = gc_ijk['origin'].str.strip()
gc_ijk['destination'] = gc_ijk['destination'].str.strip()

DISAGGREGATED MX

In [40]:
import pyzipper

def make_ijk(temp_df, metric):

    ijk = pd.melt(temp_df, id_vars='origin', var_name='destination', value_name=metric)
    # ijk['metric'] = metric

    return (ijk)

# zip_path = '../inputs/agg_mx/p21_011_Time_GC_Agg.zip'
zip_path = '../inputs/agg_mx/phase_1/50_TOUCH_Option_1A.zip'

pt_ijk = []
counter = 0

trippurpose_list = ['HBW', 'EB', 'Other', 'All']
mode_list = ['Car', 'PT']

SCEN_MAP = SCENARIO_P1_MAP.copy()

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    time_csv_list = [x for x in file_list if ("Time" in x.split("/")[-1]) and (x.endswith('.csv'))]
    pt_csv_list = [x for x in file_list if ("PT" in x.split("/")[-1]) and (x.endswith('.csv'))]

    for csvf in pt_csv_list:
        slot_no = csvf.split('/')[0].split("_")[0]
        period = csvf.split('/')[-1].split(" ")[1]
        metric = csvf.split('/')[-1].split(f"{period} ")[-1].replace(".csv", "").replace("PT ", "")
        if period.upper() in PERIOD_MAP:
            period_no = PERIOD_MAP[period.upper()]
            scenario_no = "26" + str(period_no) + str(slot_no)

            with zip_ref.open(csvf) as f:
                temp_df = pd.read_csv(f, skiprows=10).rename(columns={'p/q/[val]': 'origin'})
                temp_ijk = make_ijk(temp_df, f'{scenario_no}_{metric}')
                
                if counter == 0:
                    pt_ijk = temp_ijk.copy()

                else:
                    pt_ijk = pt_ijk.merge(temp_ijk, on=['origin', 'destination'], how='outer')
                # combined_ijk = make_time_ijk(temp_df)
                # combined_ijk['scenario'] = scenario_no
                # combined_ijk['mode'] = mode
                # combined_ijk['trippurpose'] = trippurpose
                # combined_ijk['csv_no'] = csvf.split('/')[-1].split('_')[0]
                counter += 1
                
                # time_ijks.append(combined_ijk)

pt_ijk# .to_csv('../outputs/time_gc/26150_PT.csv', index=None)

,origin,destination,26150_HBW,26250_HBW,26350_HBW,26150_HBE,26250_HBE,26350_HBE,26150_HBSh,26250_HBSh,...,26350_Other GC,26150_Journey Time,26150_Distance,26150_Fare Matrix,26250_Journey Time,26250_Distance,26250_Fare Matrix,26350_Journey Time,26350_Distance,26350_Fare Matrix
0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4999.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489995,99996,99992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489996,99996,99993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489997,99996,99994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489998,99996,99995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
total_check = 0

pt_ijk['26150_tp_sum'] = 0
pt_ijk['26250_tp_sum'] = 0
pt_ijk['26350_tp_sum'] = 0

for trippurpose in ['HBW', 'HBE', 'HBSh', 'HBO', 'EB', 'NHBO']:
    total_check += pt_ijk[f'26150_{trippurpose}'].sum().item()
    pt_ijk['26150_tp_sum'] += pt_ijk[f'26150_{trippurpose}']
    pt_ijk['26250_tp_sum'] += pt_ijk[f'26250_{trippurpose}']
    pt_ijk['26350_tp_sum'] += pt_ijk[f'26350_{trippurpose}']

print(f"Sum of PT trips of all trip purpose, AM: {total_check}")
print(f"Total PT matrix sum, AM {pt_ijk['26150_Matrix'].sum().item()}")
print(f"Total PT matrix sum, IP {pt_ijk['26250_Matrix'].sum().item()*3}")
print(f"Total PT matrix sum, PM {pt_ijk['26350_Matrix'].sum().item()}")

print("==========================================")
print(f"Total PT person hours, AM: {(pt_ijk['26150_Matrix']*pt_ijk['26150_Journey Time']).sum().item()/60}")
print(f"Total PT person hours, IP: {(pt_ijk['26250_Matrix']*pt_ijk['26250_Journey Time']).sum().item()/60}")
print(f"Total PT person hours, PM: {(pt_ijk['26350_Matrix']*pt_ijk['26350_Journey Time']).sum().item()/60}")

print("==========================================")
print(f"Total PT person hours, AM: {(pt_ijk['26150_tp_sum']*pt_ijk['26150_Journey Time']).sum().item()/60}")
print(f"Total PT person hours, IP: {(pt_ijk['26250_tp_sum']*pt_ijk['26250_Journey Time']).sum().item()/60}")
print(f"Total PT person hours, PM: {(pt_ijk['26350_tp_sum']*pt_ijk['26350_Journey Time']).sum().item()/60}")

Sum of PT trips of all trip purpose, AM: 108073.06445054003
Total PT matrix sum, AM 108862.71063081453
Total PT matrix sum, IP 131298.50891526497
Total PT matrix sum, PM 94654.55005849835
Total PT person hours, AM: 96979.4499966275
Total PT person hours, IP: 33873.001864543025
Total PT person hours, PM: 80286.25532112527
Total PT person hours, AM: 96250.0858525528
Total PT person hours, IP: 32952.19301486305
Total PT person hours, PM: 79368.29717439767


In [57]:
counter = 0
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    time_csv_list = [x for x in file_list if ("Time" in x.split("/")[-1]) and (x.endswith('.csv'))]
    car_csv_list = [x for x in file_list if ("Car" in x.split("/")[-1]) and (x.endswith('.csv'))]

    for csvf in car_csv_list:
        slot_no = csvf.split('/')[0].split("_")[0]
        period = csvf.split('/')[-1].split(" ")[1]
        metric = csvf.split('/')[-1].split(f"{period} ")[-1].replace(".csv", "").replace("Car ", "")
        if period.upper() in PERIOD_MAP:
            period_no = PERIOD_MAP[period.upper()]
            scenario_no = "26" + str(period_no) + str(slot_no)

            with zip_ref.open(csvf) as f:
                temp_df = pd.read_csv(f, skiprows=10).rename(columns={'p/q/[val]': 'origin'})
                temp_ijk = make_ijk(temp_df, f'{scenario_no}_{metric}')
                
                if counter == 0:
                    car_ijk = temp_ijk.copy()

                else:
                    car_ijk = car_ijk.merge(temp_ijk, on=['origin', 'destination'], how='outer')

                counter += 1
                
                # time_ijks.append(combined_ijk)

car_ijk# .to_csv('../outputs/time_gc/50_Car.csv', index=None)

,origin,destination,26150_HBW,26250_HBW,26350_HBW,26150_HBE,26250_HBE,26350_HBE,26150_HBSh,26250_HBSh,...,26350_Distance,26150_HBW GC,26250_HBW GC,26350_HBW GC,26150_EB GC,26250_EB GC,26350_EB GC,26150_Other GC,26250_Other GC,26350_Other GC
0,1,1,215.051071,142.612061,254.601929,18.864170,28.183241,8.417687,18.009171,212.455093,...,6.398619,15.768675,15.680761,15.701199,13.512243,13.424330,13.444767,14.492330,15.795561,15.045999
1,1,10,5.339264,3.288992,1.681017,0.110942,0.122830,0.009686,0.090178,0.596170,...,27.004324,59.296017,58.871719,59.687794,49.773121,49.348824,50.164898,53.909412,59.356209,56.922630
2,1,100,0.022732,0.022256,0.008222,0.014766,0.011270,0.001489,0.011582,0.049387,...,79.040337,150.205185,139.103500,144.059540,114.494736,105.317345,110.273392,130.941772,143.459717,134.489792
3,1,101,0.014316,0.014471,0.005007,0.012615,0.009676,0.001186,0.004594,0.019761,...,80.857559,152.006638,142.587402,147.492310,117.574814,108.161377,113.066284,134.069839,146.976166,137.736755
4,1,102,0.021136,0.020912,0.006517,0.047548,0.037080,0.004416,0.004673,0.020286,...,79.069527,150.839874,139.804794,144.827881,115.365067,105.963684,110.753227,131.601456,144.146729,135.089798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489995,99996,99992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
489996,99996,99993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
489997,99996,99994,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
489998,99996,99995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [58]:
total_check = 0

car_ijk['26150_tp_sum'] = 0
car_ijk['26250_tp_sum'] = 0
car_ijk['26350_tp_sum'] = 0

for trippurpose in ['HBW', 'HBE', 'HBSh', 'HBO', 'EB', 'NHBO']:
    total_check += car_ijk[f'26150_{trippurpose}'].sum().item()
    car_ijk['26150_tp_sum'] += car_ijk[f'26150_{trippurpose}']
    car_ijk['26250_tp_sum'] += car_ijk[f'26250_{trippurpose}']
    car_ijk['26350_tp_sum'] += car_ijk[f'26350_{trippurpose}']

print(f"Sum of car trips of all trip purpose, AM: {total_check}")
print(f"Total car matrix sum, AM {car_ijk['26150_Matrix'].sum().item()}")

print(f"Total car person hours, AM: {(car_ijk['26150_Matrix']*car_ijk['26150_Time']).sum().item()/60}")
print(f"Total car person hours, IP: {(car_ijk['26250_Matrix']*car_ijk['26250_Time']).sum().item()/60}")
print(f"Total car person hours, PM: {(car_ijk['26350_Matrix']*car_ijk['26350_Time']).sum().item()/60}")

print("==========================================")
print(f"Total car person hours, AM: {(car_ijk['26150_tp_sum']*car_ijk['26150_Time']).sum().item()/60}")
print(f"Total car person hours, IP: {(car_ijk['26250_tp_sum']*car_ijk['26250_Time']).sum().item()/60}")
print(f"Total car person hours, PM: {(car_ijk['26350_tp_sum']*car_ijk['26350_Time']).sum().item()/60}")

Sum of car trips of all trip purpose, AM: 542752.408685186
Total car matrix sum, AM 542752.4086667872
Total car person hours, AM: 160532.57294425924
Total car person hours, IP: 295758.3969433584
Total car person hours, PM: 165547.59664972973
Total car person hours, AM: 160532.57294626918
Total car person hours, IP: 295758.39702172653
Total car person hours, PM: 165547.59662019028


In [95]:
sector_map = pd.read_csv('../inputs/sector_map/19_Internal_Sect_ga.ens', sep=':', skiprows=27)
sector_map.columns = ['sector', 'zone']
sector_map['sector'] = sector_map['sector'].str.split("ga").str[-1].astype(int)
sector_map['zone'] = sector_map['zone'].astype(int)
sector_map

,sector,zone
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
591,17,592
592,17,593
593,18,594
594,18,595


In [97]:
pt_ijka = pt_ijk[['origin', 'destination', '26150_Matrix', '26250_Matrix', '26350_Matrix', '26150_Journey Time', '26250_Journey Time', '26350_Journey Time']]
pt_ijka['origin'] = pt_ijka['origin'].astype(int)
pt_ijka['destination'] = pt_ijka['destination'].astype(int)
pt_ijka = pt_ijka.merge(sector_map.rename(columns={'zone': 'origin'}), on='origin').rename(columns={'sector':'origin_sector'})
pt_ijka = pt_ijka.merge(sector_map.rename(columns={'zone': 'destination'}), on='destination').rename(columns={'sector':'destination_sector'})

for period in [1,2,3]:
    pt_ijka[f'26{str(period)}50_phour'] = pt_ijka[f'26{str(period)}50_Matrix']*pt_ijka[f'26{str(period)}50_Journey Time']/60

pt_time_sector = pt_ijka.groupby(['origin_sector', 'destination_sector']).agg({'26150_Matrix': 'sum', '26250_Matrix': 'sum', '26350_Matrix': 'sum',
                                                              '26150_phour': 'sum', '26250_phour': 'sum', '26350_phour': 'sum'}).reset_index()

for period in [1,2,3]:
    pt_time_sector[f'26{str(period)}50_jtime'] = 60*pt_time_sector[f'26{str(period)}50_phour']/pt_time_sector[f'26{str(period)}50_Matrix']

pt_time_sector = pt_time_sector.fillna(0)
print(f"Total PT person hours, AM: {pt_time_sector['26150_phour'].sum().item()}")
print(f"Total PT person hours, IP: {pt_time_sector['26250_phour'].sum().item()}")
print(f"Total PT person hours, PM: {pt_time_sector['26350_phour'].sum().item()}")
print("===========================================")
print(f"Total PT person trips, AM: {pt_time_sector['26150_Matrix'].sum().item()}")
print(f"Total PT person trips, IP: {pt_time_sector['26250_Matrix'].sum().item()}")
print(f"Total PT person trips, PM: {pt_time_sector['26350_Matrix'].sum().item()}")

pt_time_sector.to_csv('../outputs/time_gc/50_PT.csv', index=None)


/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_53062/978992680.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_ijka['origin'] = pt_ijka['origin'].astype(int)
/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_53062/978992680.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_ijka['destination'] = pt_ijka['destination'].astype(int)


Total PT person hours, AM: 96250.08576808675
Total PT person hours, IP: 32952.19299525384
Total PT person hours, PM: 79368.29723364572
Total PT person trips, AM: 108073.06432224227
Total PT person trips, IP: 42743.90438075324
Total PT person trips, PM: 93673.58323356157


In [105]:
pt_ijka = pt_ijk[['origin', 'destination', '26150_HBW', '26250_HBW', '26350_HBW', '26150_HBW GC', '26250_HBW GC', '26350_HBW GC']]
pt_ijka['origin'] = pt_ijka['origin'].astype(int)
pt_ijka['destination'] = pt_ijka['destination'].astype(int)
pt_ijka = pt_ijka.merge(sector_map.rename(columns={'zone': 'origin'}), on='origin').rename(columns={'sector':'origin_sector'})
pt_ijka = pt_ijka.merge(sector_map.rename(columns={'zone': 'destination'}), on='destination').rename(columns={'sector':'destination_sector'})

for period in [1,2,3]:
    pt_ijka[f'26{str(period)}50_pgc_hbw'] = pt_ijka[f'26{str(period)}50_HBW']*pt_ijka[f'26{str(period)}50_HBW GC']/60

pt_time_sector = pt_ijka.groupby(['origin_sector', 'destination_sector']).agg({'26150_HBW': 'sum', '26250_HBW': 'sum', '26350_HBW': 'sum',
                                                              '26150_pgc_hbw': 'sum', '26250_pgc_hbw': 'sum', '26350_pgc_hbw': 'sum'}).reset_index()

for period in [1,2,3]:
    pt_time_sector[f'26{str(period)}50_hbw_gc'] = 60*pt_time_sector[f'26{str(period)}50_pgc_hbw']/pt_time_sector[f'26{str(period)}50_HBW']

pt_time_sector = pt_time_sector.fillna(0)
print(f"Total PT person GC, HBW, AM: {pt_time_sector['26150_pgc_hbw'].sum().item()}")
print(f"Total PT person GC, HBW, IP: {pt_time_sector['26250_pgc_hbw'].sum().item()}")
print(f"Total PT person GC, HBW, PM: {pt_time_sector['26350_pgc_hbw'].sum().item()}")
print("===========================================")
print(f"Total PT person trips, HBW, AM: {pt_time_sector['26150_HBW'].sum().item()}")
print(f"Total PT person trips, HBW, IP: {pt_time_sector['26250_HBW'].sum().item()*3}")
print(f"Total PT person trips, HBW, PM: {pt_time_sector['26350_HBW'].sum().item()}")
print("===========================================")
print(f"Total PT avg GC, HBW, AM: {60*pt_time_sector['26150_pgc_hbw'].sum().item()/pt_time_sector['26150_HBW'].sum().item()}")
print(f"Total PT avg GC, HBW, IP: {60*pt_time_sector['26250_pgc_hbw'].sum().item()/pt_time_sector['26250_HBW'].sum().item()}")
print(f"Total PT avg GC, HBW, PM: {60*pt_time_sector['26350_pgc_hbw'].sum().item()/pt_time_sector['26350_HBW'].sum().item()}")

pt_time_sector.to_csv('../outputs/time_gc/50_PT_HBW_GC.csv', index=None)

Total PT person GC, HBW, AM: 106420.6749737359
Total PT person GC, HBW, IP: 13359.445667547927
Total PT person GC, HBW, PM: 88922.64368237708
Total PT person trips, HBW, AM: 61322.02359939765
Total PT person trips, HBW, IP: 23555.375854281716
Total PT person trips, HBW, PM: 50593.11813634704
Total PT avg GC, HBW, AM: 104.12638271915858
Total PT avg GC, HBW, IP: 102.08710890603425
Total PT avg GC, HBW, PM: 105.4562125734963


/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_53062/971484912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_ijka['origin'] = pt_ijka['origin'].astype(int)
/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_53062/971484912.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_ijka['destination'] = pt_ijka['destination'].astype(int)


In [98]:
car_ijka = car_ijk[['origin', 'destination', '26150_Matrix', '26250_Matrix', '26350_Matrix', '26150_Time', '26250_Time', '26350_Time']]
car_ijka['origin'] = car_ijka['origin'].astype(int)
car_ijka['destination'] = car_ijka['destination'].astype(int)
car_ijka = car_ijka.merge(sector_map.rename(columns={'zone': 'origin'}), on='origin').rename(columns={'sector':'origin_sector'})
car_ijka = car_ijka.merge(sector_map.rename(columns={'zone': 'destination'}), on='destination').rename(columns={'sector':'destination_sector'})

for period in [1,2,3]:
    car_ijka[f'26{str(period)}50_phour'] = car_ijka[f'26{str(period)}50_Matrix']*car_ijka[f'26{str(period)}50_Time']/60

car_time_sector = car_ijka.groupby(['origin_sector', 'destination_sector']).agg({'26150_Matrix': 'sum', '26250_Matrix': 'sum', '26350_Matrix': 'sum',
                                                              '26150_phour': 'sum', '26250_phour': 'sum', '26350_phour': 'sum'}).reset_index()

for period in [1,2,3]:
    car_time_sector[f'26{str(period)}50_jtime'] = 60*car_time_sector[f'26{str(period)}50_phour']/car_time_sector[f'26{str(period)}50_Matrix']

car_time_sector = car_time_sector.fillna(0)
print(f"Total Car hours, AM: {car_time_sector['26150_phour'].sum().item()}")
print(f"Total Car hours, IP: {car_time_sector['26250_phour'].sum().item()}")
print(f"Total Car hours, PM: {car_time_sector['26350_phour'].sum().item()}")
print("===========================================")
print(f"Total Car trips, AM: {car_time_sector['26150_Matrix'].sum().item()}")
print(f"Total Car trips, IP: {car_time_sector['26250_Matrix'].sum().item()}")
print(f"Total Car trips, PM: {car_time_sector['26350_Matrix'].sum().item()}")

car_time_sector.to_csv('../outputs/time_gc/50_Car.csv', index=None)

Total PT person hours, AM: 160532.5729442592
Total PT person hours, IP: 295758.3969433583
Total PT person hours, PM: 165547.5966497297
Total PT person trips, AM: 542752.4086667873
Total PT person trips, IP: 1461798.296399215
Total PT person trips, PM: 587436.7213429023


/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_53062/1130874991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_ijka['origin'] = car_ijka['origin'].astype(int)
/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_53062/1130874991.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_ijka['destination'] = car_ijka['destination'].astype(int)
